In [1]:
import joblib
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import pandas as pd
import tensorflow as tf

# Cargar el modelo
model1 = joblib.load('XGBoost_initial_model.pkl')

# Cargar el modelo
model2 = joblib.load('ANN_incremental_model.pkl')

# Cargar el modelo
model3 = joblib.load('Random Forest_incremental_model.pkl')

# Cargar el modelo
model4 = joblib.load('LightGBM_incremental_model.pkl')

# Definir el modelo SVM lineal en TensorFlow
class LinearSVM(tf.keras.Model):
    def __init__(self):
        super(LinearSVM, self).__init__()
        self.dense = tf.keras.layers.Dense(1, activation=None, kernel_regularizer=tf.keras.regularizers.l2(0.01))

    def call(self, inputs):
        return self.dense(inputs)

# Cargar el modelo inicial
initial_model_filename = "SVM_initial_model.pkl"
initial_model = joblib.load(initial_model_filename)

In [3]:
import joblib
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Leer el archivo CSV
data = pd.read_csv('BalanceData.csv')

# Eliminar columnas no deseadas
data.drop(columns=['trans_date_trans_time', 'transaction_time'], inplace=True)

# Codificar variables categóricas
label_encoder = LabelEncoder()
categorical_cols = ['merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job']
for col in categorical_cols:
    data[col + '_encoded'] = label_encoder.fit_transform(data[col])

# Eliminar columnas originales no numéricas y otras columnas irrelevantes
data.drop(columns=['merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num'], inplace=True)

# Dividir el dataset
X_train, X_test, y_train, y_test = train_test_split(data.drop('is_fraud', axis=1), data['is_fraud'], test_size=0.2, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_dev_imputed = imputer.transform(X_dev)
X_test_imputed = imputer.transform(X_test)


In [5]:
# Cargar modelos
model1 = joblib.load('XGBoost_initial_model.pkl')
model2 = joblib.load('ANN_incremental_model.pkl')
model3 = joblib.load('Random Forest_incremental_model.pkl')
model4 = joblib.load('LightGBM_incremental_model.pkl')

# Generar predicciones para cada modelo
y_pred1 = model1.predict(X_test_imputed)
y_pred2 = model2.predict(X_test_imputed)
y_pred3 = model3.predict(X_test_imputed)
y_pred4 = model4.predict(X_test_imputed)

# Calcular matrices de confusión
conf_matrix1 = confusion_matrix(y_test, y_pred1)
conf_matrix2 = confusion_matrix(y_test, y_pred2)
conf_matrix3 = confusion_matrix(y_test, y_pred3)
conf_matrix4 = confusion_matrix(y_test, y_pred4)

# Imprimir matrices de confusión
print("Matriz de Confusión para XGBoost:")
print(conf_matrix1)
print("\nMatriz de Confusión para ANN:")
print(conf_matrix2)
print("\nMatriz de Confusión para Random Forest:")
print(conf_matrix3)
print("\nMatriz de Confusión para LightGBM:")
print(conf_matrix4)

def evaluate_model(model, X, y, threshold=0.5):
    # Predecir probabilidades
    y_pred_proba = model.predict(X)

    # Convertir probabilidades a etiquetas binarias
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Calcular matriz de confusión
    conf_matrix = confusion_matrix(y, y_pred)

    return conf_matrix


matrix = evaluate_model(initial_model, X_dev_imputed, y_dev)
print("Matriz de Confusión para SVM lineal en TensorFlow:")
print(matrix)


Matriz de Confusión para XGBoost:
[[367067   1049]
 [     0 368982]]

Matriz de Confusión para ANN:
[[368116      0]
 [368982      0]]

Matriz de Confusión para Random Forest:
[[368058     58]
 [     0 368982]]

Matriz de Confusión para LightGBM:
[[364898   3218]
 [   207 368775]]
9214/9214 [==============================] - 16s 2ms/step
Matriz de Confusión para SVM lineal en TensorFlow:
[[     0 147067]
 [     0 147772]]
